In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from importlib import reload 

#add simulator to a path
sys.path.append('Simulator/')


import estimation_methods as em
import noise_models as nm
import utils as utl  
import simulation as sm
import analyser as anl

%load_ext autoreload
%autoreload 2   



    def estimate(self, om, mu, std, t, rng_shot = np.random.default_rng()):
        x = rng_shot.binomial(1, 1/2+1/2*np.cos(2*np.pi*om*t))
        x = 2*x-1
        mu, std = update_p(x, mu, std, t) 
        return mu, std/mu


In [8]:
def likelihood(x, om, tau, dom):
    return 1/2 + x/2*np.cos(dom * tau + om * tau)

def normal_dist(x, mu, sigma):
    return np.exp(-1/2 * (x - mu)**2 / sigma**2)/np.sum(np.exp(-1/2 * (x - mu)**2 / sigma**2))

def prob_update(x, mu, std):
    
    t = 2*np.pi * t
    denom = 1 + x*np.cos(mu*t)*np.exp(-std**2*t**2/2)
    m1 = (mu+x*(mu*np.cos(mu*t)-np.sin(mu*t)*std**2*t
             )*np.exp(-std**2*t**2/2))/denom
    m20 = std**2+mu**2
    m2 = (m20+x*((mu**2+std**2-std**4*t**2)*np.cos(mu*t
          )-2*np.sin(mu*t)*std**2*t*mu
             )*np.exp(-std**2*t**2/2))/denom
    return m1, np.sqrt(m2-m1**2)

In [ ]:
def estimate(noise,estimation_method):
    estimation_method.reset_pdf(SHOT_TIME, noise)
    for k in range(estimation_method.N):

        a = simulator.run_single_realisation([estimation_method], noise)

 
    return estimation_method.get_avg()

def get_bit(omega_est, noise, phi):
    t = phi/omega_est*1e3/2/np.pi
    prob = (1 + np.cos(2*np.pi*noise.x*t/1e3))/2

    noise.update(SHOT_TIME)
    return np.random.choice([1,0], p = [prob, 1-prob])


In [9]:

initial_mu = 0  #initial average of dom
SHOTS = 20 #numver of shots
#initialization
dom0 = -7   #system frequency detuning to be estimated 
df0 = 2    #controlled detuning 


In [42]:
dom = [dom0]
dfs = []
taus = []
n = 4   #how wide is the filter 

def get_df(sig, mu,n):
    # such that (mu+om)*tau = pi/2, in case of which p(0) = p(1)
    return n*sig/2-mu

def get_tau(sig, n):
    # such that sig*tau = pi/n, and width of the filter is sig
    return np.pi/n/sig

print(n)
dfs.append(get_df(initial_sig,initial_mu,n)) 
taus.append(get_tau(initial_sig,n))

sigs = [initial_sig]
mus = [initial_mu]

prob_exact_reg = [normal_dist(dom_grid, initial_mu, initial_sig)]
prob_approx_reg = [normal_dist(dom_grid, initial_mu, initial_sig)]

for s in range(SHOTS):
    p = likelihood(1,dfs[-1],taus[-1],dom[-1]) #probability p = p(1), p(0) = 1-p
    x = 2 * np.random.choice(2, p = (1-p,p))-1    #measureement 1 or -1

    #update the probability distribution
    prob_exact, mu, sigma = prob_update(x, dfs[-1], taus[-1], dom_grid, mus[-1], sigs[-1])     

    #get next detuning and evolution time:
    dfs.append(get_df(sigma, mu, n))  #controlled detuning 
    taus.append(get_tau(sigma,n)) #evolution time

    #save results
    prob_exact_reg.append(prob_exact)
    prob_gaussian = normal_dist(dom_grid, mu, sigma) #project it into gaussian
    prob_approx_reg.append(prob_gaussian)
    sigs.append(sigma)
    mus.append(mu)



    

4


In [ ]:
BITS_PER_REPETITIONS = 1
REPETITIONS = 2000
phis = np.linspace(0, 5*np.pi, 51)
bits_est = np.zeros((REPETITIONS, len(phis),BITS_PER_REPETITIONS))
bits_no_est = np.zeros((REPETITIONS,len(phis),BITS_PER_REPETITIONS))
SHOT_TIME = 1e4
N_SHOTS = 5
QUBIT_FREQ = 2 #MHz


total_time = (N_SHOTS + BITS_PER_REPETITIONS)*REPETITIONS * SHOT_TIME
ommin = 1/1e8
ommax = 1/1e3
N_TELEGRAPHS = 15
noise = nm.Over_f_noise(n_fluctuators = N_TELEGRAPHS, S1  = 2,
                        couplings_dispersion = 0.2, 
                        ommax = ommax , 
                        ommin = ommin, 
                        om0 = QUBIT_FREQ,
                        fluctuator_class = nm.OrnsteinUhlenbeck)


dom = []
filtered = []
N = 5
mov_avg = 0
est_raw = []
xis = []
for n in range(REPETITIONS):
    if n%100==0:
        print(n/REPETITIONS)
    for pn,phi in enumerate(phis):

        estimate()
        xis.append(noise.x)

        if len(est_raw)<N:
            mov_avg = np.average(est_raw)
        else:
            mov_avg = mov_avg + (est_raw[-1] - est_raw[-N])/N
            
        filtered.append(mov_avg)
        for b in range(BITS_PER_REPETITIONS):
            bits_no_est[n,pn,b] = get_bit(10, noise, phi)
            bits_est[n,pn, b] = get_bit(filtered[-1], noise, phi)



In [ ]:
def estimate(noise,estimation_method):
    estimation_method.reset_pdf(SHOT_TIME, noise)
    for k in range(estimation_method.N):

        a = simulator.run_single_realisation([estimation_method], noise)

 
    return estimation_method.get_avg()

def get_bit(omega_est, noise, phi):
    t = phi/omega_est*1e3/2/np.pi
    prob = (1 + np.cos(2*np.pi*noise.x*t/1e3))/2

    noise.update(SHOT_TIME)
    return np.random.choice([1,0], p = [prob, 1-prob])
